In [1]:
import numpy as np 
from keras.models import Sequential 
from keras.layers import Dense, Reshape, Flatten, Conv2D 
from keras.optimizers import Adam 
from google.colab import files 
import pandas as pd 

In [2]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Зададим глобальные параметры
parameters = {
    'path' : '/content/drive/My Drive/2021/R-Q-Learning/', # Путь к папке с весами
    'observation_size' : 80 * 80, # 'Усеченные' размеры выбора ТТ
    'action_size'  : 85, # Количество возможных вариантов действия
    'rewardDecay' : 0.95, # Коэффициент изменения награждения
    'opt_rate' : 0.001 # rate оптимизатора    
}

In [4]:
# Функция создания модели
def create_model():
  model = Sequential()
  model.add(Reshape((80, 80, 1), input_shape=(parameters['observation_size'],)))
  model.add(Conv2D(16, 8, strides=(4, 4), activation='relu'))
  model.add(Conv2D(32, 4, strides=(2, 2),activation='relu'))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(parameters['action_size'], activation='softmax'))
  opt = Adam(lr=parameters['opt_rate'])
  model.compile(loss='categorical_crossentropy', optimizer=opt)
  return model

In [ ]:
# Функция пересчета вознаграждения
def processRewards(rewardList): # 14 агентов на входе
  rewardDecayed = np.zeros_like(rewardList,dtype=np.float32) # Создаем список из нулей с размерностью равной rewardList
  tmp = 0 # Переменная для временного значения
  for t in reversed(range(0, rewardList.size)): 
    if rewardList[t] == -1: 
      for i in range(1, 7): 
        rewardList[t-i] = -0.999          
    if rewardList[t] == 1: 
      for i in range(1, 17): 
        rewardList[t-i] = 0.999
    if rewardList[t] != 0: 
      tmp = 0 
    tmp = tmp * parameters['rewardDecay'] + rewardList[t] 
    rewardDecayed[t] = tmp 
  return rewardDecayed 

In [ ]:
# Функция преобразования фрейма
def preprocessFrames(Frame): 
  Frame = Frame[35:195] 
  Frame = Frame[::2,::2, 0] 
  Frame[Frame==144] = 0 
  Frame[Frame==109] = 0 
  Frame[Frame != 0] = 1 
  return Frame.astype(np.float).ravel() # функция вернет разницу между кадрами в виде одномерного вектора

## Создание модели

In [ ]:
env = []
observation = env.reset() 
lastObservation = None 
model = create_model()

# Создадим пустые массивы под выборки
gradients = [] # Сюда будем записывать награждение за правильный шаг и противоположное значение за неправильный на текущем шаге
states = [] # Массив состояний в течение одного эпизода
rewards = [] # Массив наград в течение одного эпизода
predictions = [] # Массив, в который будем записывать получаемые на выходе модели значения
num_episode = 101000 # Номер эпизода

# СТАТИСТИКА
frameCount = 0 # Количество фреймов в эпизоде
score = 0 # Результат текущего эпизода
losses = [] # Массив, куда будем записывать ошибки
frameCounts = [] # Массив, куда будем записывать фреймы в эпизоде
averages = [] # Массив для среднего значения результата эпизода
scores = [] # Массив для сохранения результатов эпизода

model.load_weights(parameters['path']+'model'+str(num_episode)+'.h5')
while True:
  # Получаем разницу кадров  
  prepFrame = preprocessFrames(observation) 
  diffFrame = prepFrame - lastObservation if lastObservation is not None else np.zeros(parameters['observation_size']) 
  lastObservation = prepFrame 

  # Определяем наше действие
  state = diffFrame.reshape([1, diffFrame.shape[0]]) 
  pred = model.predict(state, batch_size=1).flatten() 
  predictions.append(pred) 
  action = np.random.choice(parameters['action_size'], 1, p=pred)[0] 

  # Выполняем действие
  observation, reward, done, info = env.step(3) 
  y = np.zeros(parameters['action_size']) 
  y[action] = 1 
  gradients.append(np.array(y).astype('float32') - pred) 
  

  states.append(diffFrame) 
  rewards.append(reward) 
  score += reward 
  frameCount+= 1 

  if done: # Если завершился эпизод
    num_episode += 1 

    # Обучаем модель на данных 
    gradients = np.vstack(gradients)
    rewards = np.vstack(rewards) 
    rewards = processRewards(rewards) 
    rewards = rewards / np.std(rewards - np.mean(rewards)) 
    gradients *= rewards 
    
    x_train = np.squeeze(np.vstack([states])) 
    y_train = predictions + parameters['opt_rate'] * np.squeeze(np.vstack([gradients])) 
    loss = model.train_on_batch(x_train, y_train) 
    
    losses.append(loss) # Добавялем ошибки
    frameCounts.append(frameCount) 
    scores.append(score) 
    observation = env.reset() 
    average = sum(scores)/len(scores) 
    averages.append(average) 

    # Выводим статистику за эпизод
    print('Эпизод №: %d - Коэффициэнт: %d (%.2f) - Среднее значение: %.2f - Предикт: [%.3f, %.3f] - Ошибка: %.4f - Фреймов: %d (%.1f)' \
          % (num_episode, score, np.mean(np.array(scores)[-20:]), (average), pred[0], pred[1], loss, frameCount, np.mean(np.array(frameCounts)[-20:])))
    # Обнуляем значения
    frameCount = 0 
    score = 0
    states = []
    predictions = []
    rewards = []
    gradients = []

    # Каждый 200-ый эпизод сохраняем модель
    if num_episode > 1 and num_episode % 100 == 0:
      model.save_weights(parameters['path']+'model'+str(num_episode)+'.h5')

PS: модель считает комбинации на вхождение в норму в размере 9,5 рабочих часов